In [2]:
import matplotlib.pyplot as plt
import numpy as np
from typing import Callable
%load_ext autoreload
%autoreload 2
%matplotlib qt

# Busca incremental e busca adaptativa

## $ \S 1 $ Descrição da busca incremental para encaixotamento de raízes

Suponha que queiramos encaixotar um zero de uma função contínua $ f \colon [a, b] \to \mathbb{R} $. Usando o método da **busca incremental**, começamos escolhendo um **incremento** ou **tamanho de passo** $ h > 0 $. Então calculamos os sinais de
$$
f(x_i) \quad \text{e} \quad f(x_{i+1}) \quad \text{para} \quad x_i = a+ih \quad (i = 0, 1, \dots )\,,
$$
sucessivamente.
Se 
$$
    \operatorname{sinal} f(x_i) \ne \operatorname{sinal} f(x_{i+1})
$$
para algum $ i $, então o intervalo $ [x_i,x_{i+1}] $ deve conter um zero de $ f $,
pelo teorema do valor intermediário. Caso esta condição falhe para todo $ i $, 
eventualmente teremos $ x_{i+1} > b $ e a busca terá sido inconclusiva.

# $ \S 2 $ Implementação da busca incremental

In [3]:
def incremental_search(f: Callable[[float], float], a: float, b: float,
                       h: float) -> tuple[float, float]:
    """
    Beginning with x_0 = a and x_1 = a + h and successively incrementing (or
    decrementing) both by h, returns the first pair of consecutive nodes where f
    takes on opposite signs.
    Parameters:
        * A real continuous function f.
        * The endpoints a and b of an interval [a, b] where f is defined.
        * A step size h /= 0.
    Output:
        * In case of success: updated values of a, b such that |b - a| <= h
          and f(a)f(b) < 0. In case of failure: None.
    Prints:
        * The pair a, b in case of success; or a warning message in case of
          failure.
    """
    from numpy import sign
    

    if h == 0:
        raise ValueError("0 is not a valid value for h!")
    # Initializing:
    x_0 = a
    x_1 = a + h
    f_0 = f(x_0)
    f_1 = f(x_1)
    
    while sign(f_0) == sign(f_1):
        if x_1 > b:
            return None, None
        x_0, f_0 = x_1, f_1
        x_1 += h
        f_1 = f(x_1)
        
    return x_0, x_1

__Problema 1:__ A implementação da busca incremental acima ainda funciona caso $ a > b $? Explique.

**Problema 2:** O polinômio $ x^3 - 11x^2 + 7 $ possui exatamente um zero entre $ 0 $ e $ 1 $. Encaixote este zero dentro de um intervalo de comprimento no máximo $ 10^{-3} $.

*Solução:*

⚠️ Se $ f $ não tem zeros no intervalo $ [a,b] $, obviamente a busca incremental será mal-sucedida. Por outro lado, mesmo que $ f $ tenha zeros aí, por menor que seja o valor do incremento $ h $, não há como garantir *a priori* que a busca será bem sucedida, ou que o intervalo resultante conterá um *único* zero.

⚠️ Muitas vezes precisamos encontrar *todos* os zeros de uma função. Nestes casos, se utilizarmos um incremento $ h $ grande demais, corremos o risco de pular por um número par de zeros consecutivos sem detectá-los, ou de obter um intervalo que contém um número ímpar de zeros em seu interior, mas ao final conseguir localizar apenas um deles. Por outro lado, se $ h $ for pequeno demais, gastaremos muito tempo procurando em regiões que não contêm qualquer zero.

**Problema 3:**

(a) Quantos zeros a função $ f(x) = \sin(x) - 0.999 $ tem no intervalo $ [0, 30\pi] $?

(b) Mostre que `busca_incremental` com tamanho de passo $ h = 1 $ não consegue encaixotar nenhum deles.

*Solução:*

In [ ]:
from numpy import sin, pi

## $ \S 5 $ Busca incremental adaptativa

**Definição:** As funções

\begin{alignat*}{9}
    \lfloor{\cdot}\rfloor \colon \mathbb{R} \to \mathbb{Z},
    \quad \lfloor{x}\rfloor & = \text{maior inteiro $ \le x $} \\
    \lceil{\cdot}\rceil \colon \mathbb{R} \to \mathbb{Z},
    \quad \lceil{x}\rceil & = \text{menor inteiro $ \ge x $} \\
\end{alignat*}
são chamadas de funções **chão** e **teto** respectivamente. 

Observe que estas funções são contínuas exceto nos inteiros, onde têm uma descontinuidade de salto.

**Problema 3:** Mostre que
$$
\lceil{x}\rceil =
\begin{cases}
    \lfloor{x}\rfloor + 1 & \text{se $ x \not \in \mathbb{Z} $} \\
    \lfloor{x}\rfloor & \text{se $ x \in \mathbb{Z} $}
\end{cases}
$$



**Teorema 2:** _No pior caso, a busca incremental aplicada a uma função definida no intervalo $ [a, b] $ com tamanho de passo $ h $ requer_
$$
    \boxed{n = \lfloor{\frac{b-a}{h}}\rfloor + 1 \ \text{avaliações}.}
$$

**Prova:** Suponha que a função à qual a busca foi aplicada não possua zeros em $ [a, b] $. Então precisamos avaliá-la em cada ponto $ x_i = a + ih $ tal que $ x_i \le b $, começando de $ i = 0 $. O último inteiro $ i $ que satisfaz esta condição é
$$
    \lfloor{\frac{b-a}{h}}\rfloor. \tag*{$\blacksquare$}
$$

Por exemplo, se quisermos localizar o zero de uma função definida no intervalo $ [0, 1] $ com precisão de ao menos $ 5 $ dígitos, precisamos realizar a princípio $ 10^5 $ avaliações da função. Assim, para uma tolerância pequena relativamente ao comprimento do intervalo original, o custo computacional pode ser inaceitavelmente alto.

Uma alternativa para tentar reduzir este custo é implementar uma **busca adaptativa**, em que começamos com um incremento de tamanho $ h = \frac{b-a}{10} $ e a cada passo reduzimos o seu comprimento pela metade até que ele fique menor que a tolerância desejada. O ponto crucial é que a cada vez que encontrarmos um subintervalo onde a função troca de sinal, podemos restringir nossa busca a este intervalo menor para diminuir o número de avaliações.

In [ ]:
def adaptive_search(f: Callable[[float], float], a: float, b: float,
                    h_max: float) -> tuple[float, float]:
    """
    Initially an incremental search is performed on the given interval [a, b]
    with a step size h = (b - a) / 10. If the search is successful, [a, b]
    is updated to the interval that was yielded. Otherwise it remains the same.
    In any case, we again perform an incremental search, this time with a step
    of size one-tenth the size of the previous one. This is repeated until either:
        * The search is successful and the step size h < h_max; or
        * The search is not succesful and the step size h < h_min.
    Parameters:
        * A real continuous function f.
        * The endpoints a and b of an interval [a, b] where f is defined.
        * An upper bound h_max for the desired step size.
        * A lower bound h_min such that if h becomes smaller than h_min, and no interval where f changes sign was found, the procedure terminates.
    Output:
        * In case of success: updated values of a, b such that |b - a| < h_max
          and f(a)f(b) < 0. In case of failure: None.
    Prints:
        * The pair a, b in case of success; or a warning message in case of
          failure.
    """
    from numpy import sign
    
    # Initializing:
    x_0 = a
    x_1 = b
    h =  (b - a) / 10
    
    while h >= h_max:
        x_0, x_1 = incremental_search(f, a, b, h)
        if x_0:
            a = x_0
            b = x_1
        h /= 10
    
    if x_0:
        return x_0, x_1
    else:
        print("The search was inconclusive!")
        return None, None

⚠️ Não há garantia que a busca adaptativa seja sempre mais eficiente que a busca incremental simples.

**Problema:** Modifique `busca_incremental` para criar um novo procedimento `busca_exaustiva` com os mesmos parâmetros que antes, mas que retorna uma lista contendo todos os pares $ \big(x_{i-1}, x_i\big) $ tais que ocorre uma troca de sinal no intervalo com estas extremidades, onde
$$
    x_i = a + h i \quad \text{para} \quad i = 0,1,\dots,N - 1 =\left\lfloor{\frac{b-a}{h}}\right\rfloor \quad\text{e} \quad x_N = b.
$$

*Solução:*